In [ ]:
import sys
sys.path.append("../")

In [ ]:
import torch
from samplers.ParallelTempering import parallel_tempering_step
import numpy as np
from utils import GMM, quadratic_function, MC_estimate_true_expectation, relative_mae, plot_contours
from mmd import MMD_loss
from tqdm import tqdm

device = torch.device('cuda')
torch.set_default_tensor_type('torch.FloatTensor')

In [ ]:
dim = 2
n_mixes = 40
loc_scaling = 40.0  # scale of the problem (changes how far apart the modes of each Guassian component will be)
log_var_scaling = 1.0 # variance of each Gaussian

torch.manual_seed(0)  # seed of 0 for GMM problem
target = GMM(dim=dim, n_mixes=n_mixes,
              loc_scaling=loc_scaling, log_var_scaling=log_var_scaling,
              device=device)

energy = lambda x: -target.log_prob(x)

plotting_bounds = (-loc_scaling * 1.4, loc_scaling * 1.4)

n_samples = int(1e4)
true_samples = target.sample([n_samples])
plot_contours(target.log_prob, samples=true_samples.cpu().numpy(), bounds=plotting_bounds, n_contour_levels=50, grid_width_n_points=200, device=device, plt_show=False)

true_expectation = MC_estimate_true_expectation(true_samples.cpu(), quadratic_function)
print("True Expectation:", true_expectation.cpu().numpy())

In [ ]:
SEED = 1
np.random.seed(SEED)
torch.manual_seed(SEED)

In [ ]:
num_samples = 10000
num_chains = 10
step_size = 1e-1
num_leapfrog_steps_per_hmc_step = 10
num_hmc_steps = 20

# (inverse) temperature schedule
num_temperatures = 5
inv_temperatures = [0.0 for _ in range(num_temperatures)]
b_min = 0.001

for r in range(num_temperatures):
    t = r / (num_temperatures - 1)
    inv_temperatures[r] = b_min ** t
print("Temperatures:", [round(1.0/inv_t, 2) for inv_t in inv_temperatures])

x = [torch.zeros([num_chains, dim], device=device) for _ in range(num_temperatures)]

energy = lambda x: -target.log_prob(x)

samples = []
for _ in tqdm(range(num_samples//num_chains)):
    chains = parallel_tempering_step(x, num_temperatures, energy, step_size, num_leapfrog_steps_per_hmc_step, num_hmc_steps, inv_temperatures, device)
    samples.append(chains[0].detach())
samples = torch.cat(samples, dim=0)
plot_contours(target.log_prob, samples=samples.cpu().numpy(), bounds=plotting_bounds, n_contour_levels=50, grid_width_n_points=200, device=device, plt_show=True)

expectation = MC_estimate_true_expectation(samples.cpu(), quadratic_function)
print("Expectation:", expectation.cpu().numpy())

print("Relative MAE:", relative_mae(true_expectation, expectation).item())

mmd_loss=MMD_loss(kernel_num=5)
mmd=mmd_loss.forward(source=samples.cpu(), target=true_samples.cpu())
print("MMD:", mmd.item())